In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)

    print('acc: %.3f, roc-auc: %.3f' % (acc,auc))

    return acc, auc

In [2]:
label = pd.read_excel('./data/subject_info.xlsx', index_col = 'Subject_ID')
data = pd.read_excel('./data/data_species_7278_preprocessed.xlsx', index_col = 0)

In [3]:
label.groupby('Group').size()

Group
HS               30
Healthy         127
MP               40
Stage_0          27
Stage_III_IV     54
Stage_I_II       69
dtype: int64

In [4]:
mp = loadmap('./CRC_stage.mp')

## Model-1: Healthy vs. MP:

In [5]:
stage = 'MP'
model_dir = './Stage_%s_model' % stage

sld = label[label.Group.isin(['Healthy', stage])].index
dfy = label.loc[sld][['Group']]
dfx = data.loc[sld]
dfx = np.log(dfx + 1e-8)
dfy.shape, dfx.shape

((167, 1), (167, 7278))

In [7]:
X = mp.batch_transform(dfx.values, scale_method = 'standard')
Y = pd.get_dummies(dfy['Group']).values

100%|##########| 167/167 [00:17<00:00,  9.79it/s]


In [8]:
dfy.shape, dfx.shape

((167, 1), (167, 7278))

In [9]:
class_ = pd.get_dummies(dfy['Group']).columns
class_

Index(['Healthy', 'MP'], dtype='object')

In [ ]:
outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128]#5 repeats random seeds 8, 16, 32, 64, 128

each_fold_results = []
run_all_res = []
for repeat_seed in repeat_seeds: 
    
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(X, Y[:,0])

    
    run_one_res = []
    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 50, gpuid = 1, conv1_kernel_size = 13,
                                           batch_size = 2, verbose = 0) #
        clf.fit(trainX, trainY ) 

        ## save model for interpretation
        if repeat_seed == 8:
            clf._model.save(os.path.join(model_dir, 'seed_%s_fold_%s.h5' % (repeat_seed, i)))
    
        y_true = testY[:,1] #Stage
        y_score = clf.predict_proba(testX)[:,1]
        
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        
        acc, auc = score(dfr)
        
        run_one_res.append(dfr)
        
        each_fold_results.append([auc, i, repeat_seed])
        
    run_all_res.append(pd.concat(run_one_res))


 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='1')
acc: 0.941, roc-auc: 0.923

 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='1')
acc: 0.765, roc-auc: 0.615

 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='1')
acc: 0.882, roc-auc: 1.000

 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='1')
acc: 0.882, roc-auc: 0.981

 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='1')
acc: 0.882, roc-auc: 0.731

 input train and test X shape is (150, 86, 85, 5), (17, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epoch

In [ ]:
results = []
for dfr, repeat_seed in zip(run_all_res,repeat_seeds):

    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])

    sensitivity = tp / sum([tp, fn])
    specificity = tn / sum([tn, fp])

    prc_auc = prc_auc_score(y_true, y_score)
    roc_auc = roc_auc_score(y_true, y_score)

    precision = tp / sum([tp, fp])
    recall =  tp / sum([tp, fn]) #equals to sensitivity

    res = {
           'repeat_seed':repeat_seed,

           'accuracy':acc, 

           'prc_auc':prc_auc, 
           'roc_auc':roc_auc,

           'sensitivity': sensitivity, 
           'specificity': specificity,

           'precision':precision,
           'recall':recall,
           'F1': 2*precision*sensitivity/(precision+sensitivity)
          }
    
    results.append(res)

In [ ]:
pd.DataFrame(results).roc_auc.mean()

In [ ]:
pd.DataFrame(results)

In [ ]:
pd.DataFrame(results).to_csv('./results/performance_results_%s.csv' % stage)